In [0]:
!pip install sqlalchemy
!pip install setproctitle
!pip install ray[rllib]  # also recommended: ray[debug]
!pip uninstall -y pyarrow

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd  /content/drive/My\ Drive/Year\ 4\:\ Synthesis/NS162\:\ Statistical\ Mechanics/Renormalization\ Research\ Project/renormalization/supervised_convnet/t_2.269/3x3/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Year 4: Synthesis/NS162: Statistical Mechanics/Renormalization Research Project/renormalization/supervised_convnet/t_2.269/3x3


In [2]:
%%capture
import train, frozen
import sys
sys.path.insert(0, "../../")
import supervised_convnet
import pickle
from collections import defaultdict
import numpy as np
import time
import torch

mode = "run"
run_mode = "unfrozen_convolution_many_channels"
save_loops = 50
use_cuda = True
results = []
conv_params = {}
filename = ""
def save_progress(results, conv_params, filename, epoch = 500):
    new_results = {}
    new_results["best_val_acc_hist"] = results
    new_results["conv_params"] = conv_params
    new_results["annotation"] = "bias + weights histogram saved"
    new_results["epoch"] = epoch
    # new_results["first_epo    ch_validate_accuracy_list"] = first_epoch_validate_accuracy_list
    with open(filename, "wb") as handle:
        pickle.dump(new_results, handle, protocol = pickle.HIGHEST_PROTOCOL)
"""
Default activation function: sigmoid
"""

# GPU for use in colab
def model_to_cuda(model):
    if use_cuda and torch.cuda.is_available():
        model = model.cuda()
    return model
  
if mode == "run":
    hidden_size = 10
    out_channels = 1
    num_hidden_layers = 1
    conv_params = defaultdict(list)

    if run_mode == "unfrozen_convolution":
        """
        """
        out_channels = 1
        filename = "unfrozen_convolution.pl"
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    activation_func = "sigmoid", num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
            conv_params["weight"].append(param_dict["conv1.weight"])
            conv_params["bias"].append(param_dict["conv1.bias"])
    elif run_mode == "unfrozen_convolution_relu":
        """
        """
        out_channels = 1
        filename = "unfrozen_convolution_relu.pl"
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        results = []
        first_epoch_validate_accuracy_list = []
        for _ in range(500):
            model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    first_activation = "tanh", activation_func = "relu",
                    num_hidden_layers = num_hidden_layers, seed = time.time() + _)
            model = model_to_cuda(model)
            best_val_acc, param_dict = train.trainer(model = model, batch_size = 86, betas0= 1-0.0018179320494754046, betas1=1- 0.001354073715524798,
                                                    lr= 0.004388469485690077, n_epochs= 250, train_size= 4277, weight_decay= 2.346792703571872e-05, use_cuda = use_cuda)
            results.append(best_val_acc)
            conv_params["weight"].append(param_dict["conv1.weight"])
            conv_params["bias"].append(param_dict["conv1.bias"])
            if (_ % save_loops) == (0):
                save_progress(results, conv_params, filename, _)
    elif run_mode == "frozen_convolution_no_center":
        """
        """
        out_channels = 1
        filename = "frozen_convolution_no_center.pl"
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "omit", activation_func = "sigmoid", num_hidden_layers = num_hidden_layers)
            model = model_to_cuda(model)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
            if (_ % save_loops) == (0):
                save_progress(results, conv_params, filename, _)
    elif run_mode == "frozen_convolution_no_center_relu":
        """
        """
        filename = "frozen_convolution_no_center_relu.pl"
        out_channels = 1
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        results = []
        for _ in range(500):
            model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "omit", first_activation = "tanh",
                    activation_func = "relu", num_hidden_layers = num_hidden_layers)
            model = model_to_cuda(model)
            best_val_acc, param_dict = train.trainer(model = model, batch_size = 100, betas0=1- 0.15428681703555264, betas1=1-1e-5,
                                                    lr= 0.021919796543990972, n_epochs= 250, train_size= 5000, weight_decay= 0.0008230830669560186, use_cuda = use_cuda)
            # conv_params["weight"].append(param_dict["conv1.weight"])
            conv_params["bias"].append(param_dict["conv1.bias"])
            results.append(best_val_acc)
            if (_ % save_loops) == (0):
                save_progress(results, conv_params, filename, _)
    elif run_mode == "frozen_convolution_with_center":
        """
        """
        filename = "frozen_convolution_with_center.pl"
        out_channels = 1
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "keep", activation_func = "sigmoid", num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
    elif run_mode == "frozen_convolution_with_center_relu":
        """
        """
        filename = "frozen_convolution_with_center_relu.pl"
        out_channels = 1
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "keep", activation_func = "relu", num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
    elif run_mode == "frozen_convolution_pretrained_relu":
        """
        """
        filename = "frozen_convolution_pretrained_relu.pl"
        out_channels = 1
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        results = []
        first_epoch_validate_accuracy_list = []
        for _ in range(500):
            model = frozen.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "pre_trained", first_activation = "tanh",
                    activation_func = "relu", num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict = train.trainer(model = model, lr = 0.005)
            results.append(best_val_acc)
    elif run_mode == "unfrozen_convolution_many_channels":
        """
        check to make sure that out_channels below is not 1
        """
        out_channels = 3
        filename = "unfrozen_convolution_{}_channels.pl".format(out_channels)
        print("filename", filename)
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3,
                    hidden_size = hidden_size, out_channels = out_channels,
                    activation_func = "sigmoid", num_hidden_layers = num_hidden_layers,
                    seed = time.time() + _)
            best_val_acc, param_dict = train.trainer(model = model, batch_size = 100, betas0= 1-0.08600510236382716, betas1= 1-0.148443948081689,
                                                    lr= 0.0007024370411205373, n_epochs= 250, train_size= 5000, weight_decay= 1.000000000004125e-05, use_cuda = use_cuda)
            results.append(best_val_acc)
            conv_params["weight"].append(param_dict["conv1.weight"])
            conv_params["bias"].append(param_dict["conv1.bias"])
            if (_ % save_loops) == (0):
                save_progress(results, conv_params, filename, _)
    elif run_mode == "unfrozen_convolution_many_channels_relu":
        """
        check to make sure that out_channels below is not 1
        """
        out_channels = 2
        filename = "unfrozen_convolution_{}_channels_relu.pl".format(out_channels)
        print("filename", filename)
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3,
                    hidden_size = hidden_size, out_channels = out_channels,
                    first_activation = "tanh", activation_func = "relu",
                    num_hidden_layers = num_hidden_layers, seed = time.time() + _)
            best_val_acc, param_dict = train.trainer(model = model)
            results.append(best_val_acc)
            conv_params["weight"].append(param_dict["conv1.weight"])
            conv_params["bias"].append(param_dict["conv1.bias"])
    elif run_mode == "unfrozen_convolution_many_channel_many_hidden":
        """
        check to make sure that out_channels, num_hidden_layers above is not 1
        """
        out_channels = 2
        num_hidden_layers = 2
        filename = "unfrozen_convolution_{}_channels_{}_hidden.pl".format(out_channels, num_hidden_layers)
        print("filename", filename)
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        for _ in range(500):
            model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels, num_hidden_layers = num_hidden_layers)
            best_val_acc, param_dict, first_epoch_validate_accuracy = train.trainer(model = model)
            results.append(best_val_acc)
            

save_progress(results, conv_params, filename)



Uncorrelated [[ 1  1  1 -1 -1 -1  1 -1 -1]
 [ 1  1  1 -1 -1 -1 -1 -1 -1]
 [ 1  1  1 -1  1  1  1  1  1]
 [ 1  1 -1  1 -1 -1 -1 -1 -1]
 [ 1  1 -1  1 -1 -1 -1 -1 -1]
 [ 1  1  1  1 -1 -1 -1 -1  1]
 [ 1  1 -1 -1 -1 -1  1  1  1]
 [ 1 -1 -1 -1 -1 -1  1  1  1]
 [ 1  1  1 -1 -1 -1  1  1 -1]]
Correlated [[ 1  1 -1  1 -1 -1 -1 -1  1]
 [ 1  1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  1 -1 -1 -1  1  1  1  1]
 [-1 -1 -1 -1 -1 -1  1  1  1]
 [ 1 -1 -1 -1 -1  1  1  1  1]
 [-1 -1 -1 -1 -1  1  1  1  1]
 [-1 -1 -1 -1 -1  1  1 -1 -1]
 [-1 -1 -1 -1 -1 -1  1  1  1]
 [-1 -1  1 -1 -1 -1 -1 -1  1]]


ModuleNotFoundError: ignored

In [0]:
i = 0